# IMI Big Data Competition - Anti-money Laundring 
# Data Preprocessing

This script preprocesses the AML data abd output a merged data

written by C.-H. Eric Chang, updated Feb, 2021

In [1]:
# import required libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## Inspect data
### Customer data set

The customer data set contains customer-level data on retail accounts for which we have labels

Columns:
* The first five columns correspond to information given by the customer
* The variable "rating" means risk of AML: 1 = low risk, 2 = medium risk, 3 = potentially high risk
* Columns beginning with “PCD” or “SRV” correspond to ‘product’ or ‘account’ information (e.g., PCD_MOR: how many accounts of type “MOR” does this customer have)

In [2]:
# Load the customer training data set
df_cust = pd.read_parquet("cust_train.parquet")
df_cust

,customer_id_mskd,jurisdiction_code,client_type_aml,industry_code_aml,occupation_code_aml,country_of_domicile_aml,occupation_status_code_aml,customer_status_aml,export_ts,primary_ownership_flag,...,PCD_TED,SRV_FLG,SRV_FSL,SRV_ILC,SRV_LOC,SRV_NLG,SRV_NSL,SRV_TRF,PRD_INFO_AVAIL,rating
0,df39588796bc20fb01d282213200c944ad38caa06a91a7...,CA03,Individual,None,108,CA,None,Active,2020-05-13,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,3.0
1,d56f5632aa40d2661dcc5419def6ede2ed4cca24bd76cd...,CA03,Individual,None,342,CA,None,Active,2020-05-13,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,3.0
2,a86efc98b6e70bbb37c12024304f3d7c9eac9e4ecd9559...,CA03,Individual,None,E902,CA,E902,Active,2020-05-13,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,3.0
3,24f154d9bb05a18ba723830b6c41d6eb71a67f14ca7325...,CA03,Individual,None,94,CA,None,Active,2020-05-13,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,3.0
4,bdb58c996ed0b5c9f81461caa8efe914f712e22ffaa6ac...,CA03,Individual,None,256,CA,None,Active,2020-05-13,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4464,4e5114e6ee4f4fd9e6008b8617997397e1efd8d3c0ed4c...,CA03,Individual,None,E902,CR,E902,Active,2020-05-13,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2.0
4465,bb74589e5e2582fb9407fb8e2810473abbf738458ed968...,CA03,Individual,None,212,CA,None,Active,2020-05-13,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2.0
4466,c648253e0be1e7e26fd6acbd8566aa23ef686e76de73eb...,CA03,Individual,None,256,CA,None,Active,2020-05-13,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2.0
4467,5f3b76c7308cbd6000231d88f214cbbb949c611b0d1c03...,CA08,Individual,None,E904,CA,E904,Active,2020-05-13,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2.0


In [3]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4469 entries, 0 to 4468
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   customer_id_mskd            4469 non-null   object        
 1   jurisdiction_code           4469 non-null   object        
 2   client_type_aml             4469 non-null   object        
 3   industry_code_aml           0 non-null      object        
 4   occupation_code_aml         4341 non-null   object        
 5   country_of_domicile_aml     4463 non-null   object        
 6   occupation_status_code_aml  2064 non-null   object        
 7   customer_status_aml         4469 non-null   object        
 8   export_ts                   4469 non-null   datetime64[ns]
 9   primary_ownership_flag      4376 non-null   object        
 10  relationship_type           4376 non-null   object        
 11  PCD_CDA                     1617 non-null   float64     

In [4]:
# Remove columns that only contain NaNs (i.e., the number of NaNs is equal to the number of rows)
nanCols = df_cust.columns[df_cust.isna().sum()==len(df_cust)] # columns that only contain NaNs
df_cust.drop(nanCols, axis=1, inplace=True) # remove these columns
# df_cust.info()

In [5]:
# # Frequency table of AML rating grouped by customer information
# for i in ['jurisdiction_code','client_type_aml','occupation_code_aml','country_of_domicile_aml','occupation_status_code_aml','customer_status_aml']:
#     print(df_cust.groupby([i,'rating'])['customer_id_mskd'].count())
#     print()

In [6]:
df_cust.describe()

,PCD_CDA,PCD_CRC,PCD_LLC,PCD_MOR,PCD_SAV,PCD_SDB,PCD_TED,rating
count,1617.000000,1411.000000,845.000000,326.000000,1003.000000,131.000000,860.000000,4469.000000
mean,1.255411,1.546421,1.216568,1.251534,1.557328,1.022901,1.320930,2.019691
std,0.700740,0.969649,0.583510,0.747044,1.233859,0.150161,0.737664,0.778555
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
75%,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,3.000000
max,10.000000,9.000000,9.000000,9.000000,18.000000,2.000000,9.000000,3.000000


### Transaction data set

The transaction data set contains monthly aggregated transaction data on retail accounts in the customer data set

Columns:
* "in_amt" and "out_amt": The total volume entering and exiting each product for each customer
* "in_cnt" and "out_cnt": The total count of transactions over which that volume was distributed

In [7]:
# load the transaction training data set:
df_trsact = pd.read_parquet("transaction_train.parquet")
df_trsact

,customer_id_mskd,month,in_amt,in_cnt,out_amt,out_cnt,trsactn_type
1,4eb76d305d32c1d00cc0d8850abe45ffc49f51f08324ee...,2020-02,150.0,1.0,1600.00,2.0,cash
5,510ad85cdbe68ad98edaf81cbc2fda6fc218d1c4acc53b...,2019-04,400.0,1.0,1000.00,2.0,cash
6,8115b4ba5e267184283a64117e9664deb55159c1bd4e2d...,2019-05,NaN,NaN,500.00,1.0,cash
7,6c854026065a868b6134d04d906dcfa9af7278ff834b0c...,2019-06,NaN,NaN,315.00,6.0,cash
8,d8cc95339794bf3f84f941b1eba6266d58af1dc7e65178...,2020-01,1200.0,2.0,NaN,NaN,cash
...,...,...,...,...,...,...,...
92882,e81c38e11c65f8eb3513c2da404909133d57e4a66edb81...,2019-10,NaN,NaN,1055.56,3.0,debit
92883,9c2cde673de7afd9d1b60618cc6fc51542ac95dfea9cbf...,2019-06,NaN,NaN,471.81,4.0,debit
92884,e6ffe16501c564ffdca94214fdffe78e1bfb09783edfc0...,2019-08,NaN,NaN,3152.45,53.0,debit
92886,b66bb4312fd69679c650bb30fa7d317867a6cb0bf2aed0...,2019-11,NaN,NaN,1974.98,37.0,debit


In [8]:
df_trsact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69534 entries, 1 to 92887
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id_mskd  69534 non-null  object 
 1   month             69534 non-null  object 
 2   in_amt            37726 non-null  float64
 3   in_cnt            37726 non-null  float64
 4   out_amt           60818 non-null  float64
 5   out_cnt           60818 non-null  float64
 6   trsactn_type      69534 non-null  object 
dtypes: float64(4), object(3)
memory usage: 4.2+ MB


In [9]:
df_trsact.describe()

,in_amt,in_cnt,out_amt,out_cnt
count,3.772600e+04,37726.000000,6.081800e+04,60818.000000
mean,6.033623e+03,2.745375,3.993000e+03,15.606038
std,8.579920e+04,3.808859,4.613588e+04,25.377144
min,1.000000e-02,1.000000,1.000000e-02,1.000000
25%,2.423775e+02,1.000000,2.479475e+02,2.000000
50%,9.312300e+02,2.000000,7.992000e+02,5.000000
75%,3.004220e+03,3.000000,2.194942e+03,18.000000
max,8.333333e+06,172.000000,5.134000e+06,748.000000


In [10]:
# # Frequency table of the transaction type variable in the transaction data
# df_trsact.groupby('trsactn_type').count()

## Merge data

In [11]:
# Unique customer id in the customer data set
custID_cust_uniq = df_cust['customer_id_mskd'].unique()
# number of uniqe customer id in the customer training data set
print('Number of unique customer IDs in the customer data:', str(len(custID_cust_uniq)))

# Unique customer id in the transaction training data set
custID_trsact_uniq = df_trsact['customer_id_mskd'].unique()
# number of uniqe customer id in the transaction training data set
print('Number of unique customer IDs in the transaction data:', str(len(custID_trsact_uniq)))

# The transaction training data set has less unique customer id than the customer training data set
# np.isin(custID_cust_uniq, custID_trsact_uniq).sum()

Number of unique customer IDs in the customer data: 4469
Number of unique customer IDs in the transaction data: 2827


### Remove customers without transaction data when merging data sets

Inner join the customer data set and the transaction data set

In [12]:
df_merge = df_cust.merge(df_trsact, on='customer_id_mskd')
df_merge

# Keep customers without transaction data in the merged data
# df_merge_left = df_cust.merge(df_trsact, on='customer_id_mskd', how='left')

,customer_id_mskd,jurisdiction_code,client_type_aml,occupation_code_aml,country_of_domicile_aml,occupation_status_code_aml,customer_status_aml,export_ts,primary_ownership_flag,relationship_type,...,PCD_SDB,PCD_TED,PRD_INFO_AVAIL,rating,month,in_amt,in_cnt,out_amt,out_cnt,trsactn_type
0,df39588796bc20fb01d282213200c944ad38caa06a91a7...,CA03,Individual,108,CA,None,Active,2020-05-13,N,Joint,...,NaN,NaN,False,3.0,2019-08,NaN,NaN,281.81,2.0,cash
1,df39588796bc20fb01d282213200c944ad38caa06a91a7...,CA03,Individual,108,CA,None,Active,2020-05-13,N,Joint,...,NaN,NaN,False,3.0,2019-10,NaN,NaN,427.15,2.0,cash
2,df39588796bc20fb01d282213200c944ad38caa06a91a7...,CA03,Individual,108,CA,None,Active,2020-05-13,N,Joint,...,NaN,NaN,False,3.0,2019-12,NaN,NaN,103.15,1.0,cash
3,df39588796bc20fb01d282213200c944ad38caa06a91a7...,CA03,Individual,108,CA,None,Active,2020-05-13,N,Joint,...,NaN,NaN,False,3.0,2020-01,NaN,NaN,304.00,1.0,cash
4,df39588796bc20fb01d282213200c944ad38caa06a91a7...,CA03,Individual,108,CA,None,Active,2020-05-13,N,Joint,...,NaN,NaN,False,3.0,2019-09,NaN,NaN,102.50,1.0,cash
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69529,c648253e0be1e7e26fd6acbd8566aa23ef686e76de73eb...,CA03,Individual,256,CA,None,Active,2020-05-13,Y,Sole,...,NaN,NaN,False,2.0,2020-03,15250.00,3.0,31934.16,56.0,visa
69530,c648253e0be1e7e26fd6acbd8566aa23ef686e76de73eb...,CA03,Individual,256,CA,None,Active,2020-05-13,Y,Sole,...,NaN,NaN,False,2.0,2019-11,6306.02,4.0,21841.57,14.0,visa
69531,c648253e0be1e7e26fd6acbd8566aa23ef686e76de73eb...,CA03,Individual,256,CA,None,Active,2020-05-13,Y,Sole,...,NaN,NaN,False,2.0,2019-07,3555.40,3.0,7212.65,16.0,visa
69532,c648253e0be1e7e26fd6acbd8566aa23ef686e76de73eb...,CA03,Individual,256,CA,None,Active,2020-05-13,Y,Sole,...,NaN,NaN,False,2.0,2019-05,27416.05,7.0,27177.09,16.0,visa


In [13]:
df_merge.info()
# df_merge.groupby('customer_id_mskd')['month'].count()

# df_trsact[df_trsact['customer_id_mskd']=='002b16545bbd16e87c66226dcbe7e0f95b25dccc33d517fc696411c53872d847']
# df_trsact.groupby('customer_id_mskd')['month'].count()

# df_cust[df_cust['customer_id_mskd']=='002b16545bbd16e87c66226dcbe7e0f95b25dccc33d517fc696411c53872d847']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69534 entries, 0 to 69533
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   customer_id_mskd            69534 non-null  object        
 1   jurisdiction_code           69534 non-null  object        
 2   client_type_aml             69534 non-null  object        
 3   occupation_code_aml         69464 non-null  object        
 4   country_of_domicile_aml     69440 non-null  object        
 5   occupation_status_code_aml  32026 non-null  object        
 6   customer_status_aml         69534 non-null  object        
 7   export_ts                   69534 non-null  datetime64[ns]
 8   primary_ownership_flag      69258 non-null  object        
 9   relationship_type           69258 non-null  object        
 10  PCD_CDA                     30189 non-null  float64       
 11  PCD_CRC                     26484 non-null  float64   

In [14]:
# Examine the unique values for each categorical variable
# list of columns that have categorical variables
cols_cat = ['jurisdiction_code','client_type_aml','occupation_code_aml','country_of_domicile_aml',
            'occupation_status_code_aml','customer_status_aml','primary_ownership_flag','relationship_type',
            'trsactn_type'] 
for col in cols_cat:
    print(col,':')
    print(df_merge[col].unique())
    print('\n')

jurisdiction_code :
['CA03' 'CA16' 'CA08']


client_type_aml :
['Individual']


occupation_code_aml :
['108' '342' 'E902' '94' 'E904' '256' '245' '5' '184' '49' '80' '1' '208'
 'E900' '183' '313' '70' '86' 'E901' '303' '23' '289' '212' '141' '185'
 '143' '239' '280' '250' '201' '15' '149' '234' '188' '399' '345' '205'
 '100' '121' '144' '27' '132' '334' '53' '281' '138' '196' '258' '2' '62'
 '146' '125' '229' '330' '231' '363' '343' '214' '251' '179' '82' '215'
 '38' '110' '390' None '314' '117' '358' '14' '122' '104' '230' '308'
 '344' '102' '139' '397' '391' '20' '93' '18' '227' '43' '114' '349' '46'
 '4' '337' '54' '253' '268' '76' '259' '366' '207' '356' '360' '34' '173'
 '271' '377' '237' '293' '89' '123' '346' '81' '398' '392' '92' '152'
 '274' '140' '65' '135' '72' '321' '389' '148' '339' '295' '278' '326'
 '352' '279' '286' '11' '290' '79' '200' '103' '75' '254' '42' '55' '127'
 '85' '78' '67' '312' '69' '71' '261' '32' '175' '353' '115' '21' '252'
 '318' '137' '45' '324' '287'

### The following categorical variables have less than (or equal to) 3 levels across customers:
* jurisdiction_code
* client_type_aml 
* customer_status_aml
* primary_ownership_flag


### The grouping table shows that there could be multiple transaction data in each month for each customer

In [15]:
df_merge.groupby(['customer_id_mskd','month'])['rating'].count()

customer_id_mskd                                                  month  
002b16545bbd16e87c66226dcbe7e0f95b25dccc33d517fc696411c53872d847  2019-04    1
                                                                  2019-05    1
                                                                  2019-06    1
002cbed46139d2c4476f10cc89146aaffb58676d8ab10c76af8f8d1cf3157810  2019-04    3
                                                                  2019-05    3
                                                                            ..
fff6fab0394553b89c6ce81015aab64084f02f7d16038c2783ac711dedb14af8  2019-12    3
                                                                  2020-01    3
                                                                  2020-02    4
                                                                  2020-03    2
                                                                  2020-04    2
Name: rating, Length: 29660, dtype: int64

In [16]:
# Fill NaNs with zeros for coutinuous variables: PCD_CDA, ... PCD_TED, and in_amt, in_cnt, out_amt, out_cnt
cols_cont = ['PCD_CDA','PCD_CRC','PCD_LLC','PCD_MOR','PCD_SAV','PCD_SDB','PCD_TED',
            'in_amt','in_cnt','out_amt','out_cnt']

# Check whether there is any values of zeros in the continuous variables
# If there is no errors, we can replace missing values in these columns with zeros
# If an error occurs, it means that column has at least 1 value of zero, and we need to think whether we can replace missing values with zeros
for col in cols_cont:
    assert (df_merge['PCD_CDA']==0).sum() == 0

# Make sure that there is NO case when in_amt contains a value but in_cnt is NaN (same applies to out_amt and out_cnt)
assert ~(df_trsact['in_amt'].isna() ^ df_trsact['in_cnt'].isna()).any()
assert ~(df_trsact['out_amt'].isna() ^ df_trsact['out_cnt'].isna()).any()

# Fill missing values with zeros
for col in cols_cont:
    df_merge.fillna({col: 0}, inplace=True)

# df_merge = df_merge.fillna()
# df_merge = df_merge.fillna({'in_amt':0, 'in_cnt':0, 'out_amt':0, 'out_cnt':0})
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69534 entries, 0 to 69533
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   customer_id_mskd            69534 non-null  object        
 1   jurisdiction_code           69534 non-null  object        
 2   client_type_aml             69534 non-null  object        
 3   occupation_code_aml         69464 non-null  object        
 4   country_of_domicile_aml     69440 non-null  object        
 5   occupation_status_code_aml  32026 non-null  object        
 6   customer_status_aml         69534 non-null  object        
 7   export_ts                   69534 non-null  datetime64[ns]
 8   primary_ownership_flag      69258 non-null  object        
 9   relationship_type           69258 non-null  object        
 10  PCD_CDA                     69534 non-null  float64       
 11  PCD_CRC                     69534 non-null  float64   

In [17]:
# Save the merged data as a parquet file
df_merge.to_parquet("merge_cln.parquet")